In [ ]:
Pkg.add("Jacobi")
Pkg.add("GaussQuadrature")

In [60]:
using Jacobi

f_0 = 1
H = 1

function b_n(n::Int64)
    numerator = -n*(n+1)
    denominator = (n+2)*(n+3)
    return numerator/denominator
end

function M_in(size::Int64)
    fullMatrix = zeros(size,size)
    for i=0:size-1
        for j=0:size-1
            if i==j
                fullMatrix[i+1, j+1] = 2/(2*i+1) + (b_n(i)*b_n(i))*(2/(2*i+5))
            elseif i==j+2
                fullMatrix[i+1, j+1] = b_n(j)*(2/(2*j+5))
            elseif i==j-2
                fullMatrix[i+1, j+1] = b_n(i)*(2/(2*j+5))
            end
        end
    end
    return fullMatrix
end

function B_in(size::Int64)
    fullMatrix = zeros(size,size)
    for i=0:size-1
        for j=0:size-1
            if i==j
                fullMatrix[i+1,j+1]=2/(2*i+1)
            elseif i==j+2
                fullMatrix[i+1,j+1]=b_n(j)*2/(2*j+5)
            end
        end
    end
    return fullMatrix
end

function L_in(N::Function, size::Int64)
    fullArray = zeros(size,size)
    values = zgj(size, 0, 0)
    weights = wgj(values, 0, 0)
    for i=0:size-1
        for j=0:size-1
            sumTotal = 0
            for k=0:size-1
                dp_i = djacobi(values[k+1],i,0,0)+b_n(i)*djacobi(values[k+1],i+2,0,0)
                dp_j = djacobi(values[k+1],j,0,0)+b_n(j)*djacobi(values[k+1],j+2,0,0)
                newPiece = weights[k+1]*(f_0*f_0/(H*H*N(values[k+1])*N(values[k+1])))*dp_i*dp_j
                sumTotal = sumTotal + newPiece
            end
            fullArray[i+1,j+1] = sumTotal
        end
    end
    return fullArray
end

L_in (generic function with 1 method)

In [73]:
function u_g_rhs(u_bar::Function, size::Int64)
    fullArray = zeros(size)
    values = zgj(size, 0, 0)
    weights = wgj(values, 0, 0)
    for i =0:size-1
        sumTotal = 0
        for k=0:size-1
            p_n = jacobi(values[k+1], i,0,0)
            sumTotal = sumTotal + weights[k+1]*p_n*u_bar(values[k+1])
        end
        fullArray[i+1] = sumTotal
    end
    return fullArray
end

function u_galerkin(u_bar::Function, size::Int64)
    u_lhs = M_in(size)
    u_rhs = u_g_rhs(u_bar, size)
    return inv(u_lhs)*u_rhs
end

function p_function(size::Int64, value::Float64)
    fullArray = zeros(size)
    for i=0:size-1
        fullArray[i+1] = jacobi(value,i,0,0)+b_n(i)*jacobi(value,i+2,0,0)
    end
    return fullArray
end

function dy_vartheta_bar(N::Function, du_bar::Function, value::Float64)
    return -(f_0*f_0/(H*H*N(value)*N(value)))*du_bar(value)
end

function dy_psi(u_bar::Function, size::Int64)
    return -u_galerkin(u_bar, size)
end

function dq_galerkin(u_bar::Function, du_bar::Function, N::Function, size::Int64)
    r_piece1 = L_in(N, size)*dy_psi(u_bar, size)
    r_piece2 = dy_vartheta_bar(N,du_bar,1.0)*p_function(size,1.0)
    r_piece3 = dy_vartheta_bar(N,du_bar,0.0)*p_function(size,0.0)
    lhs = B_in(size)
    rhs = r_piece1+r_piece2-r_piece3
    return inv(lhs)*rhs
end

dq_galerkin (generic function with 1 method)

In [74]:
using Jacobi

function N_constant(value::Float64)
    return 1
end

function z_linear(value::Float64)
    return value
end

function dz(value::Float64)
    return 1
end 
l = L_in(N_constant, 5)
dpsi = dy_psi(z_linear, 5)
#p_function(5,1.0)
#dy_vartheta_bar(N_constant, dz,1.0)*p_function(5,1.0)

#u_galerkin(z_linear,5)
dq_galerkin(z_linear, dz, N_constant, 5)

5-element Array{Float64,1}:
  0.0     
 -3.73749 
 -3.28125 
 -6.51572 
 -0.984375